In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def trindex(returns, startingvalue=1):
    '''
    Takes a series of returns (%) and a starting value (default=1) and returns
    a series of index values 
    '''
    ln_returns = [x/100+1 for x in returns]
    
    price = []
    p = startingvalue

    for r in ln_returns:                    #calculate price of asset over time
        p = p * r
        price.append(p)
        
    return price

In [ ]:
def maxdd(date, returns):
    '''
    Takes a list of dates and a list of total return index values as inputs
    and returns a dataframe that contains the top drawdowns, along with 
    the start date, trough date, length of time from peak to trough 'DURATION',
    and the length of time it took to recover 'RECOVERY'
    '''
    
    price = trindex(returns)

    RESULTS = []

    for i in range(len(price)-1):

        if price[i] > price[i+1]:                #Finds peak index

            try:
                newpeak = [n for n, p in enumerate(price[i+1:]) if p>=price[i]][0]  #Looks for newpeak
            except IndexError:
                newpeak = len(price[i+1:])

            trough = min(price[i:(i+newpeak)])                              #Finds trough date
            troughi = price[i:newpeak+i].index(min(price[i:newpeak+i]))     #Finds trough index

            DD = round(((trough - price[i]) / price[i])*100, 2)           #Calculates DD from peak

            Peak = date[i]

            Trough = date[i+troughi]

            Duration = round((troughi/12), 2)

            Recovery = round((newpeak - troughi)/12, 2) if price[i] > price[i+newpeak] else None

            RESULTS.append([Peak,Trough,DD,Duration,Recovery])

    RESULTS = sorted(RESULTS, key=lambda x: x[2])        #Sorts results by DD

    TOPDD = []

    for d in RESULTS:                                   #Iterates through drawdown list to find
        if not any(d[1] in t for t in TOPDD):           #the largest DD at each unique trough date
            TOPDD.append(d)


    return pd.DataFrame(np.array(TOPDD),
                         columns=['PEAK','TROUGH','DRAWDOWN','DURATION','RECOVERY']).head(25) #Creates a DataFrame of results